In [2]:
from bs4 import BeautifulSoup as bsoup #need bs for that sweet, sweet web scraping
from urllib.request import urlopen #need to get url
import requests
from datetime import date #need to pull the current date
import calendar #calendar can find the name of the month
import string
import markovify
import random

headers = {'User-Agent':'Mozilla/5.0'}

In [3]:
#find today's date and fetch the wiki page
today = date.today()
wikiUrl = "https://en.wikipedia.org/wiki/"+calendar.month_name[today.month]+"_"+str(today.day)
print("Now running for ", wikiUrl) #just checking

wikiPage = requests.get(wikiUrl)
wikiSoup = bsoup(wikiPage.text, "html.parser")

#gets the unordered list for events that day
events_li = wikiSoup.find_all("ul")[1].find_all("li")

Now running for  https://en.wikipedia.org/wiki/June_12


In [4]:
dayUrl = "http://www.onthisday.com/today/events.php"
page = requests.get(dayUrl)
daySoup = bsoup(page.text, "html.parser")
dayList = daySoup.findAll("li", { "class" : "event-list__item" })
specials = daySoup.find_all("p") #some events are in paragraph tags
dayList.extend(specials[2:len(specials)-1])

In [5]:
#lists for the event dates and the events themselves
dates = [] #the isolated dates
events = [] #events

#iterate through all events in the list
for event in events_li:
    #split them to separate date and description
    temp = event.get_text().split("–")
    #add date to dates
    dates.append(temp[0])
    #take a slice of the rest and join it to take care of the '–' char elsewhere
    events.append("".join(temp[1::]))
    #this line would take out the titles leading each line
    #events.append("".join("".join(temp[1::]).split(":")[1::]))

In [6]:
#insert events from on this day
for event in dayList:
    #get the string of the event and strip the leading date or range of dates
    eventString = event.get_text().lstrip(string.digits+string.whitespace + '[-]')
    #get a string of the rest and add period if there isn't one for markovify
    if not eventString[len(eventString)-1] == ".":
        eventString += "."
    events.append(" " + eventString)

In [10]:
#time to get into real markov stuff
textModel = markovify.Text("".join(events))

#complete_model = markovify.Text("".join(comp))

#pick a random date from the list on the page
eventDate = random.choice(dates).strip() + ": "

#this prints out sentences for the split up events
print("SHORT SENTENCE")
for i in range(5):
    print(eventDate + textModel.make_short_sentence(140-len(eventDate)))

SHORT SENTENCE
910: Anti-apartheid activist and ANC leader Nelson Mandela is sentenced to death for crimes he had committed during his 13-year rule.
910: Bryan Allen wins the second Navy Bill, which calls for 50,000 volunteers to stop Federates from taking over his state.
910: Los Angeles Lakers beat New Jersey Nets, 4 games to 2. th Tony Awards: The Book of Mormon & War Horse win.
910: An election takes place in Nigeria which is capable of selectively absorbing carbon dioxide, is created.
910: Connie Mack named Honorary Manager of the 101st Airborne Division secure the town of Carentan.
